In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from time import time

# set seed for the random number generations (to be able to replicate the results)
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [2]:
# loading and transforming the data

# transform
transform = transforms.Compose([transforms.ToTensor(), #transforms image into colour matrices with values ranging from 0 -1 (becomes a Torch Tensor)
                              transforms.Normalize((0.5,), (0.5,)), # normalizes tensor
                              ])

# loading the data
trainset = datasets.EMNIST('\Datasets\MNIST\Trainset', download=True, split='letters', train=True, transform=transform)
valset = datasets.EMNIST('\Datasets\MNIST\Valset', download=True, split='letters', train=False, transform=transform)

# creating the batches 
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True) 
test_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True) 

In [3]:
# defining the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,5,3) # for convolution: imput size has to match number of channels for the image
        self.conv2 = nn.Conv2d(5,10,3)
        self.lin1 = nn.Linear(250, 27) # 5760 is calculated by: print(x.view(64, -1).shape)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(64, -1) # transforms to 2D. Returns a new tensor with the same data as the self tensor but of a different shape.Keep 64 batches, calculate / infer other dimension
        x = self.lin1(x) # requires 2D input, no relu required as log_softmax is performed
        return F.log_softmax(x, dim = 1)
        
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1))
  (lin1): Linear(in_features=250, out_features=27, bias=True)
)


In [4]:
# defining the optimization method
learning_rate = 0.003
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [5]:
# setting the image to the correct size
images, labels = next(iter(train_loader))
#images = images.view(images.shape[0], -1)
print(images.shape)
# images = images.transpose(1,3)

# criterion = nn.NLLLoss()
# logps = model(images) #log probabilities
# loss = criterion(logps, labels) #calculate the NLL loss

torch.Size([64, 1, 28, 28])


In [25]:
# # core training process (NN iterates over the training set and updates the weights))
# time0 = time()
# epochs = 15
# for e in range(epochs):
#     running_loss = 0
#     for images, labels in train_loader:
#         # Training pass
#         optimizer.zero_grad() # sets all optimized gradients to zero
        
#         output = model(images)
#         #loss = criterion(output, labels)
#         loss = F.nll_loss(output, labels)
        
#         #This is where the model learns by backpropagating
#         loss.backward()
        
#         #And optimizes its weights here
#         optimizer.step()
        
#         running_loss += loss.item()
#     else:
#         print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))
# print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: -0.03855772840862091
Epoch 1 - Training loss: -0.1866747986792754
Epoch 2 - Training loss: -0.3034884336208686
Epoch 3 - Training loss: -0.3119990795086592
Epoch 4 - Training loss: -0.3169930079120856
Epoch 5 - Training loss: -0.320683460411353
Epoch 6 - Training loss: -0.32358690515542643
Epoch 7 - Training loss: -0.3259819572629073
Epoch 8 - Training loss: -0.32756597068829413
Epoch 9 - Training loss: -0.3292321242812352
Epoch 10 - Training loss: -0.33007866805944686
Epoch 11 - Training loss: -0.3491408895682066
Epoch 12 - Training loss: -0.4213013410644654
Epoch 13 - Training loss: -0.44293490533645336
Epoch 14 - Training loss: -0.4480436066327951

Training Time (in minutes) = 22.55935230652491


In [26]:
# correct_count, all_count = 0, 0
# for images,labels in test_loader:
#   for i in range(len(labels)):
# #     img = images[i].view(1, 784)
#     with torch.no_grad():
#         logps = model(images)

    
#     ps = torch.exp(logps)
#     probab = list(ps.numpy()[0])
#     pred_label = probab.index(max(probab))
#     true_label = labels.numpy()[i]
#     if(true_label == pred_label):
#       correct_count += 1
#     all_count += 1

# print("Number Of Images Tested =", all_count)
# print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 20800

Model Accuracy = 0.04745192307692308


In [6]:
# training the neural network
# core training process (NN iterates over the training set and updates the weights))
# time0 = time()
epochs = 15
def train(epochs):
    #for e in range(epochs):
#     time0 = time()
        running_loss = 0
        correct = 0
        for images, labels in train_loader:
            # Training pass
            optimizer.zero_grad() # sets all optimized gradients to zero

            output = model(images)
            
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).sum()
            
            #loss 
            loss = F.nll_loss(output, labels)

            #This is where the model learns by backpropagating
            loss.backward()

            #And optimizes its weights here
            optimizer.step()

            running_loss += loss.item()
        else: 
            print("Epoch {} - Training loss: {:.4f} Train Accuracy: ({:.2f}%)".format(
                i+1, running_loss/len(train_loader),
                100. * correct / len(train_loader.dataset)))
#     print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [7]:
test_losses = []
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for images, labels in test_loader:
      output = model(images)
      test_loss += F.nll_loss(output, labels, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(labels.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print("Test los: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
# test()

In [8]:
time0 = time()
test()
for i in range(epochs):
  train(i)
  test()
print("\nTraining Time (in minutes) =",(time()-time0)/60)

C:\Users\Lina_\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test los: 3.3048, Accuracy: 131/20800 (0.63%)
Epoch 1 - Training loss: 1.1113 Train Accuracy: (67.78%)
Test los: 0.5470, Accuracy: 17394/20800 (83.62%)
Epoch 2 - Training loss: 0.4819 Train Accuracy: (85.02%)
Test los: 0.4553, Accuracy: 17955/20800 (86.32%)
Epoch 3 - Training loss: 0.4082 Train Accuracy: (87.22%)
Test los: 0.4041, Accuracy: 18272/20800 (87.85%)
Epoch 4 - Training loss: 0.3759 Train Accuracy: (88.25%)
Test los: 0.3751, Accuracy: 18386/20800 (88.39%)
Epoch 5 - Training loss: 0.3556 Train Accuracy: (88.79%)
Test los: 0.3639, Accuracy: 18440/20800 (88.65%)
Epoch 6 - Training loss: 0.3415 Train Accuracy: (89.22%)
Test los: 0.3501, Accuracy: 18575/20800 (89.30%)
Epoch 7 - Training loss: 0.3301 Train Accuracy: (89.53%)
Test los: 0.3484, Accuracy: 18576/20800 (89.31%)
Epoch 8 - Training loss: 0.3233 Train Accuracy: (89.69%)
Test los: 0.3488, Accuracy: 18565/20800 (89.25%)
Epoch 9 - Training loss: 0.3159 Train Accuracy: (89.91%)
Test los: 0.3521, Accuracy: 18517/20800 (89.02%)


In [21]:
# Print shape of images at different points in the model
model = nn.Conv2d(1,5,3)
output_pooled = F.max_pool2d(model(images),2)
print(output_pooled.shape)

model2 = nn.Conv2d(5,10,3)
output2_pooled = F.max_pool2d(model2(output_pooled),2)
print(output2_pooled.shape)

print(output2_pooled.view(64, -1).shape)

torch.Size([64, 5, 13, 13])
torch.Size([64, 10, 5, 5])
torch.Size([64, 250])


In [9]:
# save the model
torch.save(model, './03_model.pt') 
#  save the trained model